In [1]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    return str_text

In [3]:
# read_file('./moby_dick_four_chapters.txt')

In [7]:
# read_file('./melville-moby_dick.txt')

In [4]:
import spacy

In [6]:
nlp = spacy.load('en_core_web_sm',disable=['parser','tagger','ner'])

In [8]:
nlp.max_length = 1198623

In [9]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [10]:
d = read_file('./moby_dick_four_chapters.txt')

In [11]:
tokens = separate_punc(d)

C:\Users\PC\anaconda3\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [13]:
len(tokens)

11338

In [14]:
# 25 words ---> network predict #26

In [15]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    
    text_sequences.append(seq)

In [16]:
type(text_sequences)

list

In [17]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [18]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [19]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [20]:
from keras.preprocessing.text import Tokenizer

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [22]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [23]:
sequences[0]

[956,
 14,
 263,
 51,
 261,
 408,
 87,
 219,
 129,
 111,
 954,
 260,
 50,
 43,
 38,
 314,
 7,
 23,
 546,
 3,
 150,
 259,
 6,
 2713,
 14,
 24]

In [24]:
sequences[1]

[14,
 263,
 51,
 261,
 408,
 87,
 219,
 129,
 111,
 954,
 260,
 50,
 43,
 38,
 314,
 7,
 23,
 546,
 3,
 150,
 259,
 6,
 2713,
 14,
 24,
 957]

In [25]:
# Sayıların herbiri aslında her kelime için ayrı bir ID oluşturuyor.

In [27]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
314 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2713 : interest
14 : me
24 : on


In [28]:
tokenizer.word_counts

OrderedDict([('call', 27),
             ('me', 2471),
             ('ishmael', 133),
             ('some', 758),
             ('years', 135),
             ('ago', 84),
             ('never', 449),
             ('mind', 164),
             ('how', 321),
             ('long', 374),
             ('precisely', 37),
             ('having', 142),
             ('little', 767),
             ('or', 950),
             ('no', 1003),
             ('money', 120),
             ('in', 5647),
             ('my', 1786),
             ('purse', 71),
             ('and', 9646),
             ('nothing', 281),
             ('particular', 152),
             ('to', 6497),
             ('interest', 24),
             ('on', 1716),
             ('shore', 26),
             ('i', 7150),
             ('thought', 676),
             ('would', 702),
             ('sail', 104),
             ('about', 1014),
             ('a', 10377),
             ('see', 416),
             ('the', 15540),
             ('watery', 26),
  

In [29]:
vocabulary_size = len(tokenizer.word_counts)

In [30]:
vocabulary_size

2718

In [31]:
type(sequences)

list

In [32]:
import numpy as np

In [33]:
sequences = np.array(sequences)

In [34]:
sequences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

In [35]:
from keras.utils import to_categorical

In [38]:
X= sequences[:,:-1]

In [39]:
y = sequences[:,-1]

In [40]:
y = to_categorical(y,num_classes=vocabulary_size+1)

In [41]:
seq_len = X.shape[1]

In [42]:
X.shape

(11312, 25)

In [43]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [44]:
def create_model(vocabulary_size,seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len))
    model.add(LSTM(seq_len*2,return_sequences=True))
    model.add(LSTM(seq_len*2))
    model.add(Dense(seq_len*2,activation='relu'))
    
    model.add(Dense(vocabulary_size,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    model.summary()
    
    return model

In [45]:
model = create_model(vocabulary_size+1,seq_len)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            67975     
                                                                 
 lstm (LSTM)                 (None, 25, 50)            15200     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 2719)              138669    
                                                                 
Total params: 244594 (955.45 KB)
Trainable params: 244594 (955.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
from pickle import dump,load

In [47]:
model.fit(X,y,batch_size=128,epochs=2,verbose=1)

Epoch 1/2
89/89 [==============================] - 7s 29ms/step - loss: 7.0023 - accuracy: 0.0337
Epoch 2/2
89/89 [==============================] - 3s 33ms/step - loss: 6.3745 - accuracy: 0.0529


In [48]:
model.save('my_mobydick_model.h5')

C:\Users\PC\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [49]:
dump(tokenizer,open('my_simpletokenizer','wb'))

In [50]:
from keras.preprocessing.sequence import pad_sequences

In [51]:
def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
    
    output_text = []
    
    input_text = seed_text
    
    for i in range(num_gen_words):
        
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded = pad_sequences([encoded_text],maxlen=seq_len,truncating='pre')
        
        pred_word_ind = model.predict(pad_encoded,verbose=0)[0].argmax()
        
        pred_word = tokenizer.index_word[pred_word_ind]
        
        input_text += ' '+pred_word
        
        output_text.append(pred_word)
        
    return ' '.join(output_text)

In [52]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [53]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [54]:
random_seed_text = text_sequences[random_pick]

In [55]:
random_seed_text

['thought',
 'i',
 'to',
 'myself',
 'the',
 'man',
 "'s",
 'a',
 'human',
 'being',
 'just',
 'as',
 'i',
 'am',
 'he',
 'has',
 'just',
 'as',
 'much',
 'reason',
 'to',
 'fear',
 'me',
 'as',
 'i',
 'have']

In [56]:
seed_text = ' '.join(random_seed_text)

In [57]:
seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [58]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

'the the the the the the the the the the the the the the the the the the the the the the the the the'

In [59]:
# Çok fazla eğitmediğimiz için çok kötü bir sonuç verdi.

In [76]:
from keras.models import load_model

In [77]:
model = load_model('epochBIG.h5')

In [85]:
tokenizer = load(open('epochBIG','rb'))

In [86]:
generate_text(model,tokenizer,seq_len,seed_text,25)

"to be seen there was no bad olfactories my own letter was cheerily listening over his hearers who 's more can go have a wearing"